# Notes
We use SARIMAX model to predict spot price:
1. By using only the predict demand to predict future spot price 
(without using true demand, and true inter_gen)


In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import pacf

from tsa_utils import *

import warnings
warnings.filterwarnings("ignore")

from time import time
from datetime import timedelta

# show float in two decimal form
plt.style.use('ggplot')
pd.set_option('display.float_format',lambda x : '%.2f' % x)

## 1) Load dataset

In [15]:
data = pd.read_csv("../../data/all.csv")
data.tail(3)

,time,spot_price_nsw,spot_price_sa,spot_price_tas,spot_price_vic,inter_gen_nsw,inter_gen_sa,inter_gen_tas,inter_gen_vic,demand_nsw,demand_sa,demand_tas,demand_vic,period
63454,2021-08-14 23:00:00,55.64,77.76,7.63,32.26,7.69,42.04,160.30,215.20,8194,1614,1207,5204,47
63455,2021-08-14 23:30:00,52.25,76.47,7.52,25.10,8.35,38.04,167.00,226.95,8022,1573,1163,5268,48
63456,2021-08-15 00:00:00,48.69,83.68,23.87,8.73,8.07,47.47,156.92,251.75,7867,1680,1139,5244,1


## 2) Join dataset and some feature engineering

In [16]:
df = data.copy(deep=False).set_index('time').asfreq('30T')
df = data.copy(deep=False).set_index('time')
df.index = df.index.astype('datetime64[ns]')
df = df.asfreq('30T')

df = df[df.index <= "2021-08-11 23:30:00"]
df_train = df[df.index <= "2020-12-31 23:30:00"]
df_cv = df[(df.index >= "2021-01-01 00:00:00") & (df.index <= "2021-06-30 23:30:00")]
df_test = df[(df.index >= "2021-07-01 00:00:00") & (df.index <= "2021-08-11 23:30:00")]

idx_cv_start = df.reset_index()[df.index == df_cv.index[0]].index[0] # index of df(full) where cv start
idx_test_start = df.reset_index()[df.index == df_test.index[0]].index[0] # index of df(full) where test start

df.tail(3)


,spot_price_nsw,spot_price_sa,spot_price_tas,spot_price_vic,inter_gen_nsw,inter_gen_sa,inter_gen_tas,inter_gen_vic,demand_nsw,demand_sa,demand_tas,demand_vic,period
time,,,,,,,,,,,,,
2021-08-11 22:30:00,55.27,52.01,20.91,54.29,156.25,240.22,166.32,330.72,8002,1735,1338,5450,46
2021-08-11 23:00:00,51.44,48.29,13.68,49.25,150.55,228.38,165.11,320.81,7849,1654,1308,5269,47
2021-08-11 23:30:00,54.42,49.82,11.29,51.37,146.67,242.53,167.19,296.36,7795,1571,1259,5332,48


## 5) Method 2: Use **predicted** demand to predict spot price 48 period ahead
- Idea: Same ideas as the previous one but with **predicted** demand.  
- Caution: Again, this will take you more than one hour to compute.

### 5.1) Load predicted demand data

In [17]:
demand_cv_predicted = pd.read_csv('predictions/vic_demand_unknow_cv_rfr.csv')
demand_test_predicted = pd.read_csv('predictions/vic_demand_unknow_test_rfr.csv')
demand_test_predicted.tail(3)

,time,demand_vic,predicted_demand_vic
2013,2021-08-11 22:30:00,5450,5535.43
2014,2021-08-11 23:00:00,5269,5346.05
2015,2021-08-11 23:30:00,5332,5297.75


In [18]:
df['predicted_demand_vic'] = np.nan
df['predicted_demand_vic'][:idx_cv_start] = df.demand_vic[:idx_cv_start] # fill with actual demand
df['predicted_demand_vic'][idx_cv_start:idx_test_start] = demand_cv_predicted.predicted_demand_vic
df['predicted_demand_vic'][idx_test_start:] = demand_test_predicted.predicted_demand_vic
df.tail(3)

,spot_price_nsw,spot_price_sa,spot_price_tas,spot_price_vic,inter_gen_nsw,inter_gen_sa,inter_gen_tas,inter_gen_vic,demand_nsw,demand_sa,demand_tas,demand_vic,period,predicted_demand_vic
time,,,,,,,,,,,,,,
2021-08-11 22:30:00,55.27,52.01,20.91,54.29,156.25,240.22,166.32,330.72,8002,1735,1338,5450,46,5535.43
2021-08-11 23:00:00,51.44,48.29,13.68,49.25,150.55,228.38,165.11,320.81,7849,1654,1308,5269,47,5346.05
2021-08-11 23:30:00,54.42,49.82,11.29,51.37,146.67,242.53,167.19,296.36,7795,1571,1259,5332,48,5297.75


### 5.2) Predict CV period 

In [19]:
one_day = 48 # periods in one day
n = int(len(df_cv)/one_day) # number of period in one day
df['predicted_spot_price_vic_2'] = np.nan
df

,spot_price_nsw,spot_price_sa,spot_price_tas,spot_price_vic,inter_gen_nsw,inter_gen_sa,inter_gen_tas,inter_gen_vic,demand_nsw,demand_sa,demand_tas,demand_vic,period,predicted_demand_vic,predicted_spot_price_vic_2
time,,,,,,,,,,,,,,,
2018-01-01 00:00:00,88.00,103.11,90.55,90.43,0.18,60.53,121.68,146.87,7100,1398,1091,4599,1,4599.00,nan
2018-01-01 00:30:00,91.86,107.17,92.28,92.46,0.15,43.07,118.73,131.68,6974,1359,1082,4398,2,4398.00,nan
2018-01-01 01:00:00,88.83,103.31,87.53,87.62,0.13,41.67,110.48,119.98,6790,1316,1071,4238,3,4238.00,nan
2018-01-01 01:30:00,73.62,88.20,76.29,73.08,0.14,42.15,120.09,123.86,6536,1240,1067,4112,4,4112.00,nan
2018-01-01 02:00:00,71.49,85.24,75.10,70.18,0.16,38.31,114.64,132.72,6339,1194,1061,3956,5,3956.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-11 21:30:00,57.50,60.61,28.30,61.52,154.26,245.42,165.37,304.83,8288,1920,1413,5911,44,5645.38,nan
2021-08-11 22:00:00,42.11,41.88,28.01,43.00,154.28,248.05,167.41,321.83,8048,1833,1375,5695,45,5705.55,nan
2021-08-11 22:30:00,55.27,52.01,20.91,54.29,156.25,240.22,166.32,330.72,8002,1735,1338,5450,46,5535.43,nan


In [20]:
one_day = 48 # periods in one day
n = int(len(df_cv)/one_day) # number of days in cv set
start = idx_cv_start 

for i in range(n):
    # we set lower and upper boundary so outliers won't affect our model too much
    # boundaries update on every looping
    iqr = df.spot_price_vic[:start].quantile(0.75) - df.spot_price_vic[:start].quantile(0.25)
    lower = df.spot_price_vic[:start].quantile(0.25) - 1.5 * iqr
    upper = df.spot_price_vic[:start].quantile(0.75) + 1.5 * iqr
    
    y_train = df.spot_price_vic[:start].clip(lower=lower, upper=upper)
    y_test = df.spot_price_vic[start:start+one_day]
    X_train = df[['demand_vic']][:start]
    X_test = df[['predicted_demand_vic']][start:start+one_day]

    order = (1, 1, 1) # AR, I, MA
    model = SARIMAX(endog=y_train,
                    exog=X_train,
                    order=order)
    
    model_fit = model.fit(disp=False)
    
    pred_start_date = y_test.index[0]
    pred_end_date = y_test.index[-1]

    predictions = model_fit.predict(exog=X_test, start=pred_start_date, end=pred_end_date)
    
    # save predictions
    df['predicted_spot_price_vic_2'][start:start+one_day] = predictions
    
    start += one_day

In [21]:
# output predictions to csv
vic_spot_price_cv_predicted_demand_48period = df[['spot_price_vic', 'predicted_spot_price_vic_2']][idx_cv_start:idx_cv_start+n*one_day].reset_index()
vic_spot_price_cv_predicted_demand_48period.columns = ['time', 'spot_price', 'predicted_spot_price']
vic_spot_price_cv_predicted_demand_48period.to_csv('predictions/vic_spot_price_cv_predicted_demand_unknow_48period.csv', index=False, header=True)

### 5.3) Predict Test period

In [22]:
one_day = 48 # periods in one day
n = int(len(df_test)/one_day) # number of days in cv set
start = idx_test_start 

In [23]:
for i in range(n):
    # we set lower and upper boundary so outliers won't affect our model too much
    # boundaries update on every looping
    iqr = df.spot_price_vic[:start].quantile(0.75) - df.spot_price_vic[:start].quantile(0.25)
    lower = df.spot_price_vic[:start].quantile(0.25) - 1.5 * iqr
    upper = df.spot_price_vic[:start].quantile(0.75) + 1.5 * iqr
    
    y_train = df.spot_price_vic[:start].clip(lower=lower, upper=upper)
    y_test = df.spot_price_vic[start:start+one_day]
    X_train = df[['demand_vic']][:start]
    X_test = df[['predicted_demand_vic']][start:start+one_day]

    order = (1, 1, 1) # AR, I, MA
    model = SARIMAX(endog=y_train,
                    exog=X_train,
                    order=order)
    
    model_fit = model.fit(disp=False)
    
    pred_start_date = y_test.index[0]
    pred_end_date = y_test.index[-1]

    predictions = model_fit.predict(exog=X_test, start=pred_start_date, end=pred_end_date)
    
    # save predictions
    df['predicted_spot_price_vic_2'][start:start+one_day] = predictions
    
    start += one_day


In [24]:
# output predictions to csv
vic_spot_price_test_predicted_demand_48period = df[['spot_price_vic', 'predicted_spot_price_vic_2']][idx_test_start:idx_test_start+n*one_day].reset_index()
vic_spot_price_test_predicted_demand_48period.columns = ['time', 'spot_price', 'predicted_spot_price']
vic_spot_price_test_predicted_demand_48period.to_csv('predictions/vic_spot_price_test_predicted_demand_unknow_48period.csv', index=False, header=True)